![](./resources/Default_cropland_map.png)

### Content

- [Introduction](###-Introduction)
- [How to run this notebook?](###-How-to-run-this-notebook?)
- [Before you start](###-Before-you-start)
- [1. Define your region of interest](#1.-Define-your-region-of-interest)
- [2. Define your year of interest](#2.-Define-your-year-of-interest)
- [3. Set some other options](#3.-Set-some-other-options)
- [4. Generate your map](#4.-Generate-your-map)
- [5. Final notes](#5.-Final-notes)

### Introduction

This notebook contains a short demo on how to use the WorldCereal system to generate a cropland extent map for your area and season of interest.<br>
The map is generated using a default model trained by the WorldCereal consortium to distinguish cropland from all other land use.


<div class="alert alert-block alert-info">
<b>Cropland definition:</b> 
Cropland is defined here as land used for temporary crops, i.e. crops with a less-than-1-year growing cycle which must be newly sown or planted for further production after the harvest. Sugar cane, asparagus, and cassava are also considered temporary crops, even though they remain in the field for more than 1 year. This cropland definition thus excludes perennial crops as well as (temporary) pastures.
</div>

### How to run this notebook?

#### Option 1: Run on Terrascope

You can use a preconfigured environment on [**Terrascope**](https://terrascope.be/en) to run the workflows in a Jupyter notebook environment. Just register as a new user on Terrascope or use one of the supported EGI eduGAIN login methods to get started.

Once you have a Terrascope account, you can run this notebook by clicking the button shown below.

<div class="alert alert-block alert-warning">When you click the button, you will be prompted with "Server Options". Make sure to select the "Worldcereal" image here. Did you choose "Terrascope" by accident? Then go to File > Hub Control Panel > Stop my server, and click the link below once again.</div>

<a href="https://notebooks.terrascope.be/hub/user-redirect/git-pull?repo=https%3A%2F%2Fgithub.com%2FWorldCereal%2Fworldcereal-classification&urlpath=lab%2Ftree%2Fworldcereal-classification%2Fnotebooks%2Fworldcereal_default_cropland.ipynb&branch=main"><img src="https://img.shields.io/badge/Generate%20default%20cropland%20map-Terrascope-brightgreen" alt="Generate default cropland map" valign="middle"></a>


<div class="alert alert-block alert-warning">
<b>WARNING:</b> <br>
Every time you click the above link, the latest version of the notebook will be fetched, potentially leading to conflicts with changes you have made yourself.<br>
To avoid such code conflicts, we recommend you to make a copy of the notebook and make changes only in your copied version.
</div>

#### Option 2: Install Locally

If you prefer to install the package locally, you can create the WorldCereal environment using **Conda** or **pip**.

First clone the repository:
```bash
git clone https://github.com/WorldCereal/worldcereal-classification.git
cd worldcereal-classification
```
Next, install the package locally:
- for Conda: `conda env create -f environment.yml`
- for Pip: `pip install .[train,notebooks]`

### Before you start

In order to run WorldCereal crop mapping jobs from this notebook, you need to create an account on the [Copernicus Data Space Ecosystem](https://dataspace.copernicus.eu/).
This is free of charge and will grant you a number of free openEO processing credits to continue this demo.

##### Optional fix in case of issues with "proj"

Run the following cell in case you experience issues with version conflicts in proj.db further down this notebook:

In [ ]:
import os
import sys

# Set PROJ environment variables to avoid PROJ database version conflicts
# This ensures PROJ uses the database from the current conda environment
proj_path = os.path.join(sys.prefix, 'share', 'proj')
os.environ['PROJ_LIB'] = proj_path
os.environ['PROJ_DATA'] = proj_path

print(f"PROJ paths set to: {proj_path}")

### 1. Define your region of interest

The WorldCereal system is currently optimized to process <b>20 x 20 km</b> tiles.<br>
In case your selected region exceeds this area, it will be automatically split, creating multiple map generation jobs.<br>

Every CDSE user has 10,000 processing credits available each month for free.<br>
Additional credits can be purchased to support large-scale processing.

<div class="alert alert-block alert-warning">
<b>IMPORTANT NOTES ON UPSCALING:</b><br> 

- We ALWAYS recommend you to first run the model on a <b>representative set of small test areas</b> (up to 100 km²) to visually check for model performance BEFORE upscaling to large areas!!

- By default, CDSE users are limited to running 2 processing jobs in parallel. This will result in long processing times for large areas. When engaging in country-scale mapping, we therefore recommend to <b>contact the WorldCereal team</b> for dedicated support to speed up processing through [our contact form](https://esa-worldcereal.org/en/contact).

</div>

When running the code snippet below, an interactive map will be visualized.<br>

You have two options:
- Click the Rectangle button on the left hand side of the map to start drawing your region of interest. The widget will automatically store the coordinates of the last rectangle you drew on the map.
- Use the file upload button in the upper right corner to upload a GeoPackage or zipped shapefile to define your bounding box.

In [ ]:
from worldcereal.utils.map import ui_map

# Setting area limit to 2500 km²
map = ui_map(area_limit=2500)

### 2. Define your year of interest

The default WorldCereal cropland model always uses a time series of exactly one year as input for the model.

However, instead of just using a calendar year (January 1st - December 31st), we recommend to define the exact start and end date of your time series, or processing period, based on the timing of the local growing seasons.

Take the following example for Western Europe, where we typically have a winter season (s1) and summer season (s2):

<p align="center">
<img src="./resources/Cropland_inference_choose_end_date.png" width="500"/>
</p>


The best timing to start and end the time series in this case would be October (green vertical line), as for both seasons this would result in nicely capturing the growing season within the time series. It would not make sense in this case to define the year from June to June (red vertical line), as you will miss the start of the summer season and the end of the winter season in your time series.

So if you would like to map temporary crops in Western Europe for the year 2021, we would recommend to define your processing period as October 1st 2020 - September 30th 2021.

In case you do not know the typical seasonality of crops in your area of interest, you can consult the WorldCereal crop calendars using the function below.

Note that in case your area of interest is located in an extremely heterogeneous part of the world, the WorldCereal seasons cannot be retrieved at the moment. As a fall-back, please consult the [USDA crop calendars](https://ipad.fas.usda.gov/ogamaps/cropcalendar.aspx).

In [ ]:
from notebook_utils.seasons import retrieve_worldcereal_seasons

spatial_extent = map.get_extent()
seasons = retrieve_worldcereal_seasons(spatial_extent)

Execute the next cell, select a year and drag the slider to select your processing period.

<b>NOTE</b>: For cropland, we recommend you to select a full year (12 months) as processing period, as the underlying model has been trained based on 12 months of data.

In [ ]:
from notebook_utils.dateslider import date_slider

slider = date_slider()

### 3. Set processing options

Here, we provide an overview of other processing options that are available to further tune your cropland map.<br>
If you just want to use default processing options, do not change anything in this cell.<br>

Upon executing this cell, you will be prompted to provide a short descriptive name for your model run. Results will be stored in a custom folder:<br> `runs/CROPLAND_default_{name you provided}`

In [ ]:
from pathlib import Path
from notebook_utils.classifier import get_input

# Specify the name of your run and set output directory
name = get_input('model run')
output_dir = Path('./runs') / f'CROPLAND_default_{name}'
print(f"Output directory: {output_dir}")

### OPTIONAL PARAMETERS

# Choose whether or not you want to spatially clean the classification results
postprocess_enabled = True

# Choose the postprocessing method you want to use ["smooth_probabilities", "majority_vote"]
# ("smooth_probabilities will do limited spatial cleaning,
# while "majority_vote" will do more aggressive spatial cleaning,
# depending on the value of kernel_size)
postprocess_method = "majority_vote"

# Additional parameter for the majority vote method 
# (the higher the value, the more aggressive the spatial cleaning,
# should be an odd number, not larger than 25, default = 5)
postprocess_kernel = 5


### 4. Generate your map

We now have all information we need to generate our map!<br>

The next cell takes care of splitting your area of interest into small tiles (size is specified through `tile_resolution` parameter) and generate a map for each tile.<br>

You will be able to track progress through the automated reporting.<br>

The first time you run this, you will be asked to authenticate with your CDSE account by clicking the link provided below the cell.<br>

<div class="alert alert-block alert-warning">
<b>What to do in case of interruption?</b><br> 
In case processing got interrupted, just make sure to manually set `output_dir` to the directory you previously used. In this case, processing will just continue where it stopped.
</div>

In [ ]:
import json
from worldcereal.openeo.workflow_config import WorldCerealWorkflowConfig
from worldcereal.parameters import WorldCerealProductType, CropLandParameters
from notebook_utils.production import start_production_process, monitor_production_process

# Retrieve the derived season + processing windows from the slider selection
selection = slider.get_selection()
season_selection = selection  # persist for downstream inference
season_window = selection.season_window
processing_period = selection.processing_period
year = slider._current_year
season_windows = {
    year: (
        str(season_window.start_date),
        str(season_window.end_date),
    )
}

# Parameterize the workflow
workflow_builder = (
    WorldCerealWorkflowConfig.builder()
    .season_ids([year])
    .season_windows(season_windows)
    .enable_croptype_head(False) # Set to False to disable the croptype head and only run the cropland head
    .enable_cropland_head(True) # Set to True to enable the cropland head
)
workflow_builder = workflow_builder.cropland_postprocess(
    enabled=postprocess_enabled,
    method=postprocess_method,
    kernel_size=postprocess_kernel,
)
workflow_config = workflow_builder.build()

print(
    f"Processing period (auto): {processing_period.start_date} -> "
    f"{processing_period.end_date}"
)
print("Workflow overrides applied:")
print(json.dumps(workflow_config.to_dict(), indent=2))

# Retrieve the spatial extent of the map selection and set the tile resolution for processing
processing_extent = map.get_extent()
tile_resolution = 20 # in km

# Gather all inputs for production process
args = (processing_extent, processing_period, output_dir)
kwargs = dict(
    tile_resolution=tile_resolution,
    product_type=WorldCerealProductType.CROPLAND,
    workflow_config=workflow_config,
)

# Launch production process and monitor it
proc, queue, stop_event = start_production_process(args, kwargs)
status_df = monitor_production_process(proc, queue, stop_event)

Once production across your tiles is finalized, you can use the cell below to merge the different tiles together into one map.<br>

The cropland product is a raster file consisting of three bands.<br>

1. The label of the winning class (0: no cropland, 1: cropland)
2. The probability the pixel belongs to the cropland class
3. The probability the pixel belongs to the non-cropland class (other)

In [ ]:
from notebook_utils.production import merge_maps

merged_paths = merge_maps(output_dir)
merged_list = "\n".join(
    f"{name} -> {path}"
    for name, path in merged_paths.items()
)
print("Results merged:\n" + merged_list)

You can use the next cell to quickly visualize your crop type product in this notebook.

<div class="alert alert-block alert-info">
<b>Supported visualization modes:</b><br>
By default, your product is shown using matplotlib for quick visual inspection (only the classification layer).<br><br>
By setting "interactive_mode" to True, both your classification and probability layers will be visualized in an interactive ipyleaflet window. By clicking the upper-right icon, you can toggle on/off individual layers.

<b>NOTE</b> in order for the interactive mode to work in a VSCode environment, you need to switch on port forwarding for port 8889.
</div>

In [ ]:
from notebook_utils.visualization import visualize_products

visualize_products(merged_paths, interactive_mode=False)

### 5. Final notes

Both the quantity and quality of training data are main drivers affecting the quality of the cropland extent map.<br>
Using the figure below, you get a relative indication how much training data was available for training our current default cropland model:

<p align="center">
<img src="./resources/Landcover_training_data_density_PhI.png" width="700"/>
</p>

In case you own good quality reference data on land cover and/or crop types, consider contributing these data to the WorldCereal project through our [Reference Data Module](https://rdm.esa-worldcereal.org/).